# Scipy - scientific library for Python


`scipy` is buildt on top of the `numpy` framework for multidimensional arrays, and provides a large number of higher-level scientific algorithms. Some of the topics that SciPy covers are:

* Special functions ([scipy.special](http://docs.scipy.org/doc/scipy/reference/special.html))
* Integration ([scipy.integrate](http://docs.scipy.org/doc/scipy/reference/integrate.html))
* Optimization ([scipy.optimize](http://docs.scipy.org/doc/scipy/reference/optimize.html))
* Interpolation ([scipy.interpolate](http://docs.scipy.org/doc/scipy/reference/interpolate.html))
* Fourier Transforms ([scipy.fftpack](http://docs.scipy.org/doc/scipy/reference/fftpack.html))
* Signal Processing ([scipy.signal](http://docs.scipy.org/doc/scipy/reference/signal.html))
* Linear Algebra ([scipy.linalg](http://docs.scipy.org/doc/scipy/reference/linalg.html))
* Sparse Eigenvalue Problems ([scipy.sparse](http://docs.scipy.org/doc/scipy/reference/sparse.html))
* Statistics ([scipy.stats](http://docs.scipy.org/doc/scipy/reference/stats.html))
* Multi-dimensional image processing ([scipy.ndimage](http://docs.scipy.org/doc/scipy/reference/ndimage.html))
* File IO ([scipy.io](http://docs.scipy.org/doc/scipy/reference/io.html))

Each of these submodules provides a number of functions and classes that can be used to solve problems in their respective topics.


## Special functions
Scipy implements a large amount of *special functions* (Bessel function,
Airy function, orthogonal polynomials, ...) for numneric calculations. They can be used as functions within `numpy`.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.special as ss

# we plot the n\in [1..3] Legendre polynomials.
#
x = np.linspace(-1.0, 1.0, 100)

for n in range(1, 4):
    y = ss.eval_legendre(n, x)
    plt.plot(x, y, label=r"$L_%d(x)$" % n)
    
plt.legend()    

## Numerical Integration

Numerical evaluation of a function of the type

$\displaystyle \int_a^b f(x) dx$

is called *numerical integration*, or *quadature*.
`Scipy` provides many funtions for quadrature, for example the `quad` and `dblquad` for single and double integrals, respectively.

In the follwoing I show simple examples to demonstrate the basic usage:

In [ ]:
import numpy as np
import scipy.integrate as si
import scipy

# A normlised Gauss function
def gauss(x):
    factor = (1.0 / np.sqrt(2.0 * np.pi))
    expon = np.exp(-(x)**2 / 2.0)
    
    return factor * expon

# lower and upper integration limits:
x_lower = 0 # the lower limit of x
x_upper = 1 # the upper limit of x

result, abserr = si.quad(gauss, x_lower, x_upper)

print("integral value = %f; absolute error = %e" % (result, abserr))

# Also infinite limits are possible:
result, abserr = si.quad(gauss, -scipy.Inf, scipy.Inf)

print("integral value = %f; absolute error = %e" % (result, abserr))


For simple functions we can use a lambda expression:

In [ ]:
import scipy.integrate as si
import numpy as np

val, abserr = si.quad(lambda x: (np.sin(x) + np.cos(x)**2),
                      0, np.pi / 2.)

print("result  = %f +/- %e" % (val, abserr))


Two-dimensional integrals work similarily

In [ ]:
import scipy.integrate as si
import numpy as np

def integrand(x, y):
    return np.exp(-x**2 - y**2)

x_lower = 0  
x_upper = 10
y_lower = 0
y_upper = 10

# note that the y-limits depend on x in general;
# hence they need to be given as functions of x!
val, abserr = si.dblquad(integrand, x_lower, x_upper, lambda x : y_lower, lambda x: y_upper)

print(val, abserr) 

### Exercise

Integrate the two-dimensional function $f(x, y) = \sqrt{x^2 + y^2}$ over a circle with radius $r=10$ around the origin. Perform the integration in the following two ways and compare the results:
- In polar coordinates the problem reduces to the one-dimensional integration
  $$I=2\pi\int_0^{10} r^2\, {\rm d}r$$
- When implementing a two-dimensional integration it is sufficient to perform te integral in the first quadrant and to multiply the result by 4 (symmetries).  

In [ ]:
# your solution here

## Optimization / Fitting

Determining a parametric model $y = m(x; a_0, a_1, \dots a_n)$, where the $a_i$ are parameters we would like to determine) to given data points
$(x_i; y_i \pm \Delta y_i);\; i\in [1, \dots, m]$ is called *data-fitting*. Usually the measurements $y_i$ come with some errors $\Delta y_i$. `Scipy` offers several functions for data fitting and I will show you the simplest one: `curve_fit`. It determines the best fit parameters with the $\chi^2$-method, i.e. it determines best fit parameters by minimizing the expression:

$$
\chi^2 = \sum_{i=1}^n\frac{(y_i-m(x_i; a_0, a_1, \dots a_n))^2}{(\Delta y_i)^2}
$$

Please read the [curve_fit documentation](http://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html#scipy.optimize.curve_fit) on details (error handling etc.).

For demonstration purposes we perform a line fit on some fake data:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as nr
import scipy.optimize as so

# create some fake data and plot them:
x = np.random.uniform(0., 100., 100)
# The error on each point comes from a normal distribution
# with sigma = 10
y = 2. * x + 2.6 + np.random.normal(0., 10., 100)
plt.errorbar(x, y, yerr=10, fmt=".")

In [ ]:
def fit_line(x, a, b):
    return a * x + b

# now perform the fit
# Please read carefully the documentation to see how errors
# are handled. In Physics we typically give absolute errors,
# note relative ones!
popt, pcov = so.curve_fit(fit_line, x, y,
                          sigma = np.ones(len(x)) * 10,
                          absolute_sigma=True)
print(popt, pcov)
print("a = %f +/- %f" % (popt[0], np.sqrt(pcov[0][0])))
print("b = %f +/- %f" % (popt[1], np.sqrt(pcov[1][1])))

x_fit = np.linspace(0.0, 100, 100)
y_fit = fit_line(x_fit, *(popt))
plt.errorbar(x, y, yerr=10, fmt=".")
plt.plot(x_fit, y_fit, 'r-')

## Exercise:
We look at a [(fake) dataset](data/decay_data.txt) showing the rate of detection of alpha particles close to a radioactive sample, which we will call element X. The rate of detection of alpha particles is measured at 100 different times (given by the first column, in days since the start of the experiment), and for each of these times, both the rate of detection and the uncertainty in the rate of detection are given (in the second and third columns, in detections per second).

- Plot the rate of detection versus time, including error bars 

  **Hint:** This can be done with the function `plt.errorbar`.
- The rate of decay of a radioactive element is (theoretically) given by:
\begin{equation}
  R(t) = A{\rm e}^{-t/\tau},
\end{equation}
  where $A$ is the rate of decay at time $t=0$, and $\tau$ is the  *mean lifetime* of the element. Perform a fit of this function to the data and overplot the resulting best-fit function over the data. $A$ and $\tau$ are the fit parameters of your model. Is the fit good?
- We want to consider a constant background due to possible background radiation in our model:
\begin{equation}
  R_{\rm bg}(t) = A{\rm e}^{-t/\tau} + B
\end{equation}
  Fit this new model (three fit paramters!) to the data and overplot the new best-fit model to the data.
-   One way to quantify the *goodness* of a fit is to consider the reduced $\chi^2$ value of the fit, defined as:
\begin{equation}
  \chi_{\rm red}^2 = \frac{1}{N - p - 1}\sum_{i=1}^{N}~    \left(\frac{y_i - m_i}{\sigma_i}\right)^2
\end{equation}
  where $N$ is the number of datapoints, $p$ is the number of parameters for the model, $y_i$ are the data values, $m_i$ are the model values at the same positions, and $\sigma_i$ are the uncertainties on the data $y_i$. A fit is considered to be good if $\chi_{\rm red}^2 \approx 1$. Compute $\chi_{\rm red}^2$ for both models above.

In [ ]:
# your solution jere